# Geospatial analysis to solve dad's logistics problem

System workflow:

    Spreadsheet (location, address and coord.)

                  |
                  V
      ___________________________
     |                           |
     |     Distance Matrix       |                         
     |            |              | 
     |            V              |
     |  Calculate optimal route  |
     |      (solve TSP)          |              
     |___________________________|
     
                  |
                  V
            
    CSV output with ordered locations   --->  Map 
    
                  |
                  V

        Spreadsheet and print
        


The data we need:
1. Coord of each location, including depot (start and end locations), then
2. Distance Matrix
    
Having calculated the distance matrix, we can solve the TSP and VRP.

Now, there are many ways to  calculate the distance matrix. 

The first time I solved the problem, I calculated the distance matrix using a google map's API, for which although I have free credit for now, won't be free forever and the costs to calculate the distance matrix for 66 locations add up quickly. 
The reason for this is that it can calculate a maximum of 100 matrix elements per request, so for 66 locations we need 66 requests, each one corresponding to one row of the matrix, which contains 66 elements.

Thus, we ought to find another way.

Some options are:
- [This](https://developers.google.com/optimization/routing/tsp#euclid_distance)
- [Using R](https://www.rdocumentation.org/packages/raster/versions/3.3-7/topics/pointDistance)
- [Using scipy.spatial.distance_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html)
- [Using Geographic Distance Matrix Generator](https://biodiversityinformatics.amnh.org/open_source/gdmg/index.php)

The problem with all these options is that they will give us only the geodesics. As we are not flying, only the trajectories between points with roads that we can follow without breaking the law are possible.  
For this, one solution would be to integrate one (or more) of these options with information from a maps api. Then for every pair of points, we get to know all the possible trajectories between them and use this information to calulate the distance matrix.

**Update:** As it turned out, in my search for a solution I entered a habbit hole of free and open source tools starting at OpenStreetMaps. To name a few:

- OpenStreetMaps
  - PBF format
  - Osmosis (java app for processing OSM data)
  
- [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)

- Geofabrik

- Open Source Routing Machine (OSRM)

- Open Route Service (ORS)

- Leaflet

- Folium
  - https://python-visualization.github.io/folium/
  - https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/

- Leaflet Routing Machine

- Graphhopper

- Valhalla

- VROOM 
 
- routing-ortools-osrm Python package

- GPX data format

- GeoJSON data format

- R, ORS and Leaflet. ORS has libraries for many languages, including R. GeoJSON results from ORS can be fed into Leaflet to generate visualizations. But notice that OSRM can also respond with GeoJSON results and doesn't have the limitation of 50 waypoints of ORS, although apparently it doesn't have R or Python libraries. 
  - https://giscience.github.io/openrouteservice-r
/articles/openrouteservice.html
  - https://rstudio.github.io/leaflet/json.html#wo
rking-with-raw-geojsontopojson

- Python, ORS or OSRM, Leaflet and Folium to accomplish the same as the last item. ORS is nice because it has a Python library, but its limitation of 50 waypoints is a major issue. Perhaps you could implement OSRM requests in Python and visualize directions with Leaflet and Folium. Better yet, you could develop a Python library for OSRM.  Or take a look at [this](http://project-osrm.org/docs/v5.22.0/api/#introduction). You could implement a local OSRM instance, as seen [here](https://datawookie.netlify.app/blog/2017/09/building-a-local-osrm-instance/) and [here](http://blog.yatis.io/how-to-install-osrm-and-nominatim-on-ubuntu-16-02-google-maps-alternative/), with libosrm and then on top of that a Python interface to easily manipulate data and visualize routes with Leaflet and Folium.

- GeoPandas

- Nominatim (geocoding using OpenStreetMaps data)


Together, these tools constitute a complete solution for a variety of geospatial and logistical problems.

For the distance matrix, at first I implemented OSRM's solution through the HTTP interface with Python.
Later on I'll try out OSRM's C++ library [libosrm](http://project-osrm.org/docs/v5.22.0/api/#introduction) and also  ORS's solution. ORS seems interesting as it has libraries in specific languages that ease the process of making HTTP requests

I found out that the directions service from ORS has a limit of *50* waypoints, which definitely is a problem, while the same service from OSRM apparently has a limit of *169* waypoints (or something in the ballpark). Besides, as is the case with ORS, OSRM can also responds with GeoJSON, which then can easily be fed into Leaflet for visualizations. 
Please, notice that both limits are for the HTTP service. A local instance, able to respond to any number of waypoints, can be implemented.

---

### OSRM Distance Matrix

In [1]:
# OSRM Distance Matrix
#########################

# OSRM allows a max of something around 10000 elements per request. 
# I came accross a limitation. If there are more
# than 161 addresses in a request (or something in the ballpark), 
# the server responds with a 404 error.
# It has nothing to do with the number of columns per request, since 
# the problem persists for only 1 origin and 2 destinations (1x2 matrix) 
# specified with a link containing 162 or more locations. 
# Better yet. We make multiple requests for each row to build a row, then add them
# to create the matrix.
# The temporary solution implemented to overcome this limitation 
# was to make two requests for each row, each request building
# half the row. Then the rows are gradually combined to form 
# a complete distance matrix. Obviously, it won't work for more 
# than 322 or so addresses and it makes more requests than necessary
# unless there are more than 161 locations.
# It's not elegant and don't work for an arbitrary number 
# of locations, but it works for now. Later on I update it to make
# two requests per row only if there are more than 161 locations and
# to be aware of the limitation of 10000 elem/req.

import pandas as pd 
import requests
import json 
import math


def create_distance_matrix(data):
    coords = list(data['locations'].COORDENADAS)
    distance_matrix = []
    requests_row = 2     # For now.
    for row in range(len(coords)):
        matrix_row = []
        for request in range(requests_row):
            if (request == 0) and (row < len(coords) // 2):
                partial_coords = coords[0 : len(coords) // 2]
                source = row
                destinations = partial_coords
                dest_list = []
                for dest in destinations:
                    dest_list.append(partial_coords.index(dest))
                response = send_request(partial_coords, source, dest_list)
                matrix_row = build_row(response)
            elif (row < len(coords) // 2):
                partial_coords = [coords[row]] + coords[len(coords) // 2 : ]
                source = 0 
                destinations = coords[len(coords) // 2 : ]
                dest_list = []
                for dest in destinations:
                    dest_list.append(partial_coords.index(dest))
                response = send_request(partial_coords, source, dest_list)
                matrix_row += build_row(response)
            elif (request == 0):
                partial_coords = coords[0:len(coords) // 2] + [coords[row]]
                source = len(partial_coords) - 1
                destinations = coords[0 : len(coords) // 2]
                dest_list = []
                for dest in destinations:
                    dest_list.append(partial_coords.index(dest))
                response = send_request(partial_coords, source, dest_list)
                matrix_row = build_row(response)
            else:
                partial_coords = coords[len(coords) // 2 : ]
                source = row - (len(coords) - len(partial_coords))
                destinations = partial_coords
                dest_list = []
                for dest in destinations:
                    dest_list.append(partial_coords.index(dest))
                response = send_request(partial_coords, source, dest_list)
                matrix_row += build_row(response)
        distance_matrix.append(matrix_row)
    return distance_matrix 
    
    
    
def send_request(coords, sources, destinations):
    
    # Perhaps you should put this in its own function. Or not?
    coords = pd.Series(coords)
    coords = coords.map(lambda x: x.split(','))
    coords.map(lambda x: x.reverse())
    coords = list(coords.map(lambda x: ','.join(x)))
    coords_string = ';'.join(coords)
    for dest in destinations:
        destinations[destinations.index(dest)] = str(dest)
    dest_string = ';'.join(destinations)
    
    url = 'https://router.project-osrm.org/table/v1/driving/{0}?sources={1}&destinations={2}&annotations=distance'.format(coords_string, sources, dest_string)
    response = requests.post(url)
    response = json.loads(response.text) 
    return response



#def build_distance_matrix(response):
#    distance_matrix = []
#    for row in response['distances']:
#        row_list = [row[j] for j in range(len(row))]
#        distance_matrix.append(row_list)
#    return distance_matrix


def build_row(response):
    partial_row = response['distances'][0]
    return partial_row

In [2]:
# Check input data
input = pd.read_csv('data/INPUT.csv')
print(len(input))
input

143


,Escola,Endereço,Telefone,Coordenadas
0,Sub Almoxarifado Da Educacao,"R. Salvador Cosso, 72 - Jd Iraja",(16) 3916-3734,"-21.2044655,-47.799592"
1,Casa Da Ciencia,"R. Tenente Catao Roxo, 2501 – Vila Monte Alegre",(16) 2101-9308,"-21.1627246,-47.847984"
2,Carmen Aparecida de Carvalho Ramos Profa. Emei,"R. Vitor Joao Castania, 2085 - Jd. Paiva",(16) 99125-7568,"-21.1529654,-47.848402"
3,Quintino Vieira Emei,"R. Francisco Peixoto, 151 - Jd. Paiva I",(16) 3966-2159,"-21.1528639,-47.8396996"
4,Imaculado Coracao de Maria,"R. Monte Alegre, 707 – Sumarezinho",(16) 3632-5309,"-21.1630255,-47.8352311"
...,...,...,...,...
138,Vitor Youssef Darkoubi Creche Municipal,"R. Maria Ap. Do Amaral, 535 - Planalto Verde",(16) 3639-4334,"-21.1376922,-47.84173"
139,Joao Sperandio Deputado Emei,"R. Cecilio Elias Seba, 175 – D. Bernardo Jose ...",(16) 3639-1550,"-21.135063,-47.8494503"
140,Nelson Costa Dos Santos Pe. Creche Municipal,"R. Jose Roberto Rodrigues, 116 – D. Bernardo J...",(16) 3639-0447,"-21.1356488,-47.8498948"
141,Izolina Spagnul,"R. Americo Biscaro, s/n – Jd. Alexandre Balbo",(16) 36391446,"-21.12764,-47.82654"


### OR-Tools Optimization

#### Create the data

In [5]:
# An important detail. OR-Tools works over integers, 
# so there is the need to round the values of the 
# distance matrix to integers. But if some distances are small, 
# rounding can affect the solution. For this reason, 
# we can scale the distance matrix multiplying all entries 
# by a large number, like 100. This way, When we round the entries
# the rounding amount will be very small compared to the
# distances, and thus the solution won't be affected.
# If the matrix is scaled, the solution printer must be
# changed to divide the scaled route lengths by the scaling
# factor so that it displays the unscaled distances.

from __future__ import print_function
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['locations'] = pd.read_csv('./data/INPUT.csv')
    data['distance_matrix'] = scale_matrix(create_distance_matrix(data))
    #data['distance_matrix'] = create_distance_matrix(data)
    data['num_vehicles'] = 1
    data['starts'] = [0]
    data['ends'] = [len(data['locations']) - 1]
    data['routes'] = []  # The routes that will be calculated.
    return data



# Since OR-Tools works over integers, scale the distance matrix 
# by a factor of 100 and convert it to int
def scale_matrix(matrix):
    matrix = np.int_(np.array(matrix) * 100)
    return matrix

    
# Add the solution printer
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Route distance: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    
    
    
# Save routes to a list or array.
def get_routes(solution, routing, manager):
    routes = []
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes
    
    

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], 
                                           data['starts'],
                                           data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    # https://developers.google.com/optimization/routing/routing_options#local_search_options
    #search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    #search_parameters.first_solution_strategy = (
    #    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # More advanced search strategy, called guided local search, 
    # that escapes local minima.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        data['routes'] = get_routes(solution, routing, manager)
        for i, route in enumerate(data['routes']):
            print('Route', i, route)
    
    return data

#if __name__ == '__main__':
#    main()

In [6]:
data = main()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Check distance matrix

In [25]:
# Check Distance Matrix

# Trace must be 0 for a matrix where all points are sources and destinations
print(np.trace(data['distance_matrix']) == 0)
# Num of rows and columns must be the same as num of locations
print(len(data['distance_matrix']) == len(data['locations']))
print(len(data['distance_matrix'][0]) == len(data['locations']))
data['distance_matrix']

True
True
True


array([[       0, 19516930, 19406620, ..., 19439540, 20860800,        0],
       [19256580,        0,   216410, ...,   550810,  1791720, 19256580],
       [19309730,   211760,        0, ...,   269510,  2014240, 19309730],
       ...,
       [19365140,   486110,   288940, ...,        0,  1760340, 19365140],
       [20803390,  2211230,  2100920, ...,  1823890,        0, 20803390],
       [       0, 19516930, 19406620, ..., 19439540, 20860800,        0]])

#### Save routes to a list or array

As an alternative to printing the solution directly, you can save the route (or routes, for a VRP) to a list or array. This has the advantage of making the routes available in case you want to do something with them later. For example, you could run the program several times with different parameters and save the routes in the returned solutions to a file for comparison.

In [26]:
def generate_output(data):
# Generate a csv output with the locations ordered accordingly with the route. This
# can be used as input in a map application to visualize the route and in a spreadsheet
# for printing and presentation.
    route_0 = np.array(data['routes'][0])
    locations = np.array(data['locations'])
    # Order locations according with route and save to a dataframe and csv file.
    output = locations[route_0]
    output = pd.DataFrame(output)
    output.columns = data['locations'].columns
    output.to_csv('./data/OUTPUT.csv')
    return output 

out = generate_output(data)
print(len(out) == len(data['locations']))
len(out)

True


183

## Check routes distances and durations with OSRM and ORS. It's better to verify the results with two different services.  

In [39]:
# OSRM Directions
####################

import pandas as pd
import folium

def send_request(coords):
    coords = coords.map(lambda x: x.split(','))
    coords.map(lambda x: x.reverse())
    coords = list(coords.map(lambda x: ','.join(x)))
    coords_string = ';'.join(coords)
    url =  'http://router.project-osrm.org/route/v1/driving/{}?overview=full&geometries=geojson'.format(coords_string)
    response = requests.get(url)
    return response

res = send_request(out.COORDENADAS[0:100])
route = json.loads(res.content)
route

{'code': 'Ok',
 'waypoints': [{'hint': '-R4YhwIfGIdWAAAAlgAAAAAAAAAAAAAAA-JuQi4f0EIAAAAAAAAAAFYAAACWAAAAAAAAAAAAAAB3UgAAGewN_ZB6wf5s6w39AHvB_gAAbw2XH8Qf',
   'distance': 21.859168,
   'location': [-49.419239, -20.874608],
   'name': 'Avenida das Andorinhas'},
  {'hint': 'RHUTh2h1E4cNAAAALwAAAIMAAAAAAAAAcTARQe9hAEKsk7RCAAAAAA0AAAAvAAAAgwAAAAAAAAB3UgAARt0l_aZ-vf5Z3iX93369_gEAfwqXH8Qf',
   'distance': 29.257134,
   'location': [-47.85017, -21.135706],
   'name': 'Rua José Roberto Rodrigues'},
  {'hint': 'RHUTh2h1E4cNAAAALwAAAIMAAAAAAAAAcTARQe9hAEKsk7RCAAAAAA0AAAAvAAAAgwAAAAAAAAB3UgAARt0l_aZ-vf5Z3iX93369_gEAfwqXH8Qf',
   'distance': 29.257134,
   'location': [-47.85017, -21.135706],
   'name': 'Rua José Roberto Rodrigues'},
  {'hint': 'R3UTh1R1E4chAAAAJwAAAAAAAAAKAAAAB9a6QfFO1EEAAAAAOLDSQCEAAAAnAAAAAAAAAAoAAAB3UgAAwOAl_U-Bvf4W4CX9KYG9_gAAbwqXH8Qf',
   'distance': 18.154684,
   'location': [-47.84928, -21.135025],
   'name': 'Rua Cecílio Elias Seba'},
  {'hint': '1ngTh_B4E4cNAAAAeQAAADEBAAA

In [10]:
# Visualization with Leaflet and Folium
####################################
waypoints = out
coordinates = waypoints.COORDENADAS
coordinates = coordinates.map(lambda x: x.split(','))
coordinates = coordinates.map(lambda x: [float(x[0]), float(x[1])])

m = folium.Map(location=coordinates[0], tiles='cartodbpositron', zoom_start=13)

# The popup will show the ID in the coordinate list.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(coords), tooltip=folium.Tooltip(waypoints.ESCOLA[idx]),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['routes'][0]['geometry']['coordinates']]).add_to(m)

print('Distance:', round(route['routes'][0]['distance'] / 1000, 1), 'km')

print('Duration:', round(route['routes'][0]['duration'] / 3600, 1), 'hours')
m

Distance: 566.2 km
Duration: 10.7 hours


In [11]:
# Save map
m.save('./data/ribeirao_preto/map_novo.html')

---

## OpenRouteService library 
- (https://pypi.org/project/openrouteservice/) 
- (https://hub.gke.mybinder.org/user/giscience-openrouteservice-py-koi9ssto/notebooks/examples/basic_example.ipynb)

#### Geocoding

In [28]:
# ORS Geocoding 
import openrouteservice as ors
from openrouteservice import geocode 
import pandas as pd
import folium

client = ors.Client(key='5b3ce3597851110001cf624847648e3131cd43fb8ec03d97cb46b740')
addresses = list(out['ENDEREÇO'])

# Geocode and save a map for each waypoint
for addr in addresses:
    res = geocode.pelias_search(client, '{}, Bauru'.format(addresses[addresses.index(addr)]), 
                                sources=['osm'], country='BR')
    coordinates = res['features'][0]['geometry']['coordinates']
    m = folium.Map(location=list(reversed(coordinates)), tiles='openstreetmap', zoom_start=50)
    folium.Marker(location=coordinates, 
                  popup=folium.Popup('ID: {}'.format(df.Escola[addresses.index(addr)]))).add_to(m)
    folium.Circle(list(reversed(coordinates)), radius=100).add_to(m)
    #m.save('./data/bauru_schools_maps/{}'.format(df.Escola[addresses.index(addr)]))

ApiError: 401 ({'error': 'Authorization field/api_key missing in request. If you do not have a token, please sign up for one at https://openrouteservice.org'})

#### Directions

In [31]:
# ORS Directions
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf624847648e3131cd43fb8ec03d97cb46b740')
waypoints = out
coordinates = waypoints.Coordenadas
coordinates = coordinates.map(lambda x: x.split(','))
coordinates = list(coordinates.map(lambda x: [float(x[1]), float(x[0])]))

m = folium.Map(location=list(reversed(coordinates[0])), tiles='cartodbpositron', zoom_start=10)

# The popup will show the ID in the coordinate list.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)), tooltip=folium.Tooltip(waypoints.Escola[idx]),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson'
)

folium.PolyLine(locations=[list(reversed(coord))
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

print(route['features'][0]['properties']['summary'])
m

{'distance': 119005.9, 'duration': 11382.299999999997}


In [98]:
m.save('./data/cidades/sc/maps/rota_sc_setor2.html')

#### ORS Optimization

In [ ]:
# ORS Route Optimization

#coords = ((8.34234,48.23424),(8.34423,48.26424), (8.34523,48.24424), (8.41423,48.21424))
coordinates = data['locations']['Coordenadas'][0:50]
coordinates = tuple(coordinates.map(lambda x: x.split(',')).map(lambda x: (float(x[1]), float(x[0]))))

m = folium.Map(location=coordinates[0], tiles='cartodbpositron', zoom_start=13)

# The popup will show the ID in the coordinate list.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)),
             popup=folium.Popup("ID: {}".format(idx))).add_to(m)
    
routes = client.directions(
    coordinates, 
    profile='driving-car', 
    format='geojson',
    optimize_waypoints=True
)   

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           routes['features'][0]['geometry']['coordinates']]).add_to(m)

print(routes['features'][0]['properties']['summary'])
m

#### ORS Distance Matrix

In [ ]:
# ORS Distance Matrix
import numpy as np

#coords = ((8.34234,48.23424),(8.34423,48.26424), (8.34523,48.24424), (8.41423,48.21424))
coords = data['locations']['Coordenadas'][0:59]
coords = tuple(coords.map(lambda x: x.split(',')).map(lambda x: (float(x[1]), float(x[0]))))
matrix = client.distance_matrix(coords, profile='driving-car', 
                                metrics=['distance', 'duration'])

print('Durations:\n', np.array(matrix['durations']), 
      '\n\nDistances:\n', np.array(matrix['distances']))

---

## Leaflet Routing Machine

---

## VROOM (https://github.com/VROOM-Project/vroom)

---

## Testing and building the project

In [ ]:
# Installing nox
$ pip install nox

# Running tests
$ nox

# Generating documentation
$ nox -e docs

# Copy docs to gh-pages
$ nox -e docs && mv docs/_build/html generated_docs && git clean -Xdi && git checkout gh-pages

In [3]:
# A naive recursive implementation
# of 0-1 Knapsack Problem
 
# Returns the maximum value that
# can be put in a knapsack of
# capacity W
 
 
def knapSack(W, wt, val, n):
 
    # Base Case
    if n == 0 or W == 0:
        return 0
 
    # If weight of the nth item is
    # more than Knapsack of capacity W,
    # then this item cannot be included
    # in the optimal solution
    if (wt[n-1] > W):
        return knapSack(W, wt, val, n-1)
 
    # return the maximum of two cases:
    # (1) nth item included
    # (2) not included
    else:
        return max(
            val[n-1] + knapSack(
                W-wt[n-1], wt, val, n-1),
            knapSack(W, wt, val, n-1))
 
# end of function knapSack
 
 
#Driver Code
val = [1, 2]
wt = [6.9, 9.76]
W = 12000 
n = len(val)
print(knapSack(W, wt, val, n))
 
# This code is contributed by Nikhil Kumar Singh


3
